In [1]:
from sklearn import linear_model
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from group_lasso import GroupLasso
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train_dataset.csv')
train_top = pd.read_csv('train_dataset_top.csv')
train_bottom = pd.read_csv('train_dataset_bot.csv')

test = pd.read_csv('test_dataset.csv')
test_top = pd.read_csv('test_dataset_top.csv')
test_bottom = pd.read_csv('test_dataset_bot.csv')

val= pd.read_csv('val_dataset.csv')
val_top = pd.read_csv('val_dataset_top.csv')
val_bottom = pd.read_csv('val_dataset_bot.csv')

In [3]:
test.drop(test.columns[0], axis=1, inplace=True)
test.dropna(inplace=True)

train.drop(train.columns[0], axis=1, inplace=True)
train.dropna(inplace=True)

val.drop(val.columns[0], axis=1, inplace=True)
val.dropna(inplace=True)

test_top.drop(test_top.columns[0], axis=1, inplace=True)
test_top.dropna(inplace=True)

train_top.drop(train_top.columns[0], axis=1, inplace=True)
train_top.dropna(inplace=True)

val_top.drop(val_top.columns[0], axis=1, inplace=True)
val_top.dropna(inplace=True)

test_bottom.drop(test_bottom.columns[0], axis=1, inplace=True)
test_bottom.dropna(inplace=True)

train_bottom.drop(train_bottom.columns[0], axis=1, inplace=True)
train_bottom.dropna(inplace=True)

val_bottom.drop(val_bottom.columns[0], axis=1, inplace=True)
val_bottom.dropna(inplace=True)

Out of Sample R-Squared

In [4]:
def R_oss(true,pred):
    true = np.array(true)
    pred = np.array(pred).flatten()
    print(true)
    print(pred)
    numer = np.dot((true-pred),(true-pred))
    denom = np.dot(true,true)
    frac = numer/denom
    return 1-frac

Huber loss function calculation

In [5]:
def HuberLoss(true,pred,epsilon):
    true=true.reshape((true.shape[0],1))
    residual = true-pred
    huber_loss = np.where((np.abs(residual)<1),residual**2,2*epsilon*(np.abs(residual)-0.5*epsilon))
    return np.mean(huber_loss)

Accelerated Proximal Gradient (APG) Algorithm (non-GLM)

In [6]:
def S(x,mu):
    x = np.where(np.abs(x)<=mu, 0, x)
    x = np.where((np.abs(x)>mu) & (x>0), x-mu, x)
    x = np.where((np.abs(x)>mu) & (x<0), x+mu, x)
    return x

def mse_grad(theta,X,y):
    n = len(y)
    error = X@theta-y
    return (2/n)*X.T@error

def huber_grad(theta,X,y,epsilon):
    error = X @ theta - y
    delta_abs = np.abs(error)

    gradient = np.zeros_like(theta)

    mask = delta_abs <= epsilon
    indices = np.where(mask)[0]
    if np.any(delta_abs[indices] == 0):
        gradient += 0
    else:
        gradient += X[indices].T @ (error[indices] / delta_abs[indices])

    mask = delta_abs > epsilon
    indices = np.where(mask)[0]
    modifier = epsilon * np.sign(error[indices]).T @ X[indices]
    gradient = gradient+modifier.T
    gradient /= len(y)

    return gradient


def proximal_ridge_operator(theta,gamma,lamda):
    return theta/(1+(lamda*gamma))

def proximal_lasso_operator(theta,gamma,lamda):
    return lamda*S(x=theta,mu=lamda*gamma)

def proximal_elastic_operator(theta,gamma,lamda,rho):
    return (1/(1+(lamda*gamma*rho)))*S(x=theta,mu=(1-rho)*lamda*gamma)

def theta_bar(theta,gamma,grad_loss):
    return theta-gamma*grad_loss

def theta_tilde(theta_bar,gamma,lamda,model,rho=None):
    if model == 'ridge':
        return proximal_ridge_operator(theta_bar,gamma,lamda)
    elif model == 'lasso':
        return proximal_lasso_operator(theta_bar,gamma,lamda)
    elif model == 'elastic':
        return proximal_elastic_operator(theta_bar,gamma,lamda,rho)
    
def prox_grad(gamma,lamda,X,y,model,loss_type,max_iter=1000,tol=1e-5,rho=None,epsilon=1):
    K = X.shape[1]
    X = np.array(X)
    N = len(y)
    y = np.array(y).reshape((N,1))
    theta = np.zeros((K,1))
    m=0
    if loss_type == 'huber':
        for _ in np.arange(max_iter):
            theta_m = theta
            grad_loss = huber_grad(theta=theta_m,X=X,y=y,epsilon=epsilon)
            theta_b = theta_bar(theta=theta_m,gamma=gamma,grad_loss=grad_loss)
            theta_t = theta_tilde(theta_bar=theta_b,gamma=gamma,lamda=lamda,model=model,rho=rho)
            theta = theta_t + ((m/(m+3))*(theta_t-theta_m))
            m+=1
            if np.sum(np.abs(theta-theta_m)) == 0 or np.sum((theta-theta_m)**2) < np.sum(theta_m**2*tol):
                break
    elif loss_type == 'mse':
        for m in np.arange(max_iter):
            theta_m = theta
            grad_loss = mse_grad(theta=theta_m,X=X,y=y)
            theta_b = theta_bar(theta=theta_m,gamma=gamma,grad_loss=grad_loss)
            theta_t = theta_tilde(theta_bar=theta_b,gamma=gamma,lamda=lamda,model=model,rho=rho)
            theta = theta_t + ((m/(m+3))*(theta_t-theta_m))
            m+=1
            if np.sum(np.abs(theta-theta_m)) == 0 or np.sum((theta-theta_m)**2) < np.sum(theta_m**2*tol):
                break
    return theta

def fit_APG(Xtest,theta):
    return Xtest@theta

Linear Model using OLS and MSE as loss function

In [10]:
Xtrain = [train.drop(columns=['DATE','permno','RET']).values,train_bottom.drop(columns=['DATE','permno','RET']).values,train_top.drop(columns=['DATE','permno','RET']).values]
Ytrain = [train['RET'].values,train_bottom['RET'].values,train_top['RET'].values]

Xtest = [test.drop(columns=['DATE','permno','RET']).values,test_bottom.drop(columns=['DATE','permno','RET']).values,test_top.drop(columns=['DATE','permno','RET']).values]
Ytrue = [test['RET'].values,test_bottom['RET'].values,test_top['RET'].values]
i = 0
while i<3:
    model = linear_model.LinearRegression()
    model.fit(X=Xtrain[i],y=Ytrain[i])
    pred = model.predict(X=Xtest[i])
    R2 = r2_score(y_true=Ytrue[i],y_pred=pred)
    R2_OOS = R_oss(true=Ytrue[i],pred=pred)
    if i == 0:
        print(f'In sample R2 value for complete dataset is: {R2}\n')
        print(f'Out of sample R2 value for complete dataset is: {R2_OOS}\n')
    if i == 1:
        print(f'In sample R2 value for bottom performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for bottom performers of dataset is: {R2_OOS}\n')
    if i == 2:
        print(f'In sample R2 value for top performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for top performers of dataset is: {R2_OOS}\n')
        df = pd.DataFrame({'permno':test_top['permno'],'date':test_top['DATE'],'ret':test_top['RET'],'pred':pred})
        df.to_csv('OLS_MSE.csv')
    i+=1


[-0.088191  0.079484 -0.015975 ...  0.071545 -0.065069 -0.076855]
[0.19138025 0.01578328 0.20153042 ... 0.19497756 0.30559985 0.60147626]
In sample R2 value for complete dataset is: -0.518376168525575

Out of sample R2 value for complete dataset is: -0.5137208520749783

[-0.185185 -0.0625   -0.21123  ...  0.157965 -0.171607  0.001463]
[1.01884168 0.86161526 0.80864023 ... 1.0378286  1.05825458 1.01575586]
In sample R2 value for bottom performers of dataset is: -5.907567952931123

Out of sample R2 value for bottom performers of dataset is: -5.887329848358863

[ 0.074229 -0.068037 -0.001236 ...  0.152268  0.10736   0.010708]
[ 0.2180384   0.14648726  0.0863384  ... -0.21565593 -0.06696754
  0.06547352]
In sample R2 value for top performers of dataset is: -2.191340991685312

Out of sample R2 value for top performers of dataset is: -2.1504202290179615



Linear Model using OLS and MSE as loss function. Limiting training dataset to size, book-to-market, and momentum features

In [11]:
Xtrain = [train[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,train_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,train_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Ytrain = [train['RET'].values,train_bottom['RET'].values,train_top['RET'].values]

Xtest = [test[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,test_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,test_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Ytrue = [test['RET'].values,test_bottom['RET'].values,test_top['RET'].values]
i = 0
while i<3:
    model = linear_model.LinearRegression()
    model.fit(X=Xtrain[i],y=Ytrain[i])
    pred = model.predict(X=Xtest[i])
    R2 = r2_score(y_true=Ytrue[i],y_pred=pred)
    R2_OOS = R_oss(true=Ytrue[i],pred=pred)
    if i == 0:
        print(f'In sample R2 value for complete dataset is: {R2}\n')
        print(f'Out of sample R2 value for complete dataset is: {R2_OOS}\n')
    if i == 1:
        print(f'In sample R2 value for bottom performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for bottom performers of dataset is: {R2_OOS}\n')
    if i == 2:
        print(f'In sample R2 value for top performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for top performers of dataset is: {R2_OOS}\n')
        df = pd.DataFrame({'permno':test_top['permno'],'date':test_top['DATE'],'ret':test_top['RET'],'pred':pred})
        df.to_csv('OLS_MSE_3.csv')
    i+=1

[-0.088191  0.079484 -0.015975 ...  0.071545 -0.065069 -0.076855]
[0.00988984 0.01304656 0.01026158 ... 0.01019136 0.01856729 0.01154293]
In sample R2 value for complete dataset is: -0.001303388414232609

Out of sample R2 value for complete dataset is: 0.0017665913659755672

[-0.185185 -0.0625   -0.21123  ...  0.157965 -0.171607  0.001463]
[-0.01292972 -0.01284957 -0.01213476 ...  0.01477793  0.01477793
  0.01477793]
In sample R2 value for bottom performers of dataset is: 0.00025511891228668926

Out of sample R2 value for bottom performers of dataset is: 0.0031842166188748022

[ 0.074229 -0.068037 -0.001236 ...  0.152268  0.10736   0.010708]
[-0.00535936 -0.00288817 -0.00249315 ...  0.0181021   0.02045904
  0.01670873]
In sample R2 value for top performers of dataset is: -0.0017548775387934423

Out of sample R2 value for top performers of dataset is: 0.011090059339286418



Linear model using OLS and Huber as loss function

In [12]:
##############
#
#In OLS with MSE, it was found that the performance was better when using the size, book-to-market, and momentum features instead of all features.
#Thus, will not include data for any model that uses all features, but will leave code in case user is curious about performance.
#Note: In original report, published by S. Gu, B. Kelly, and D. Xiu
#
##############
from sklearn.linear_model import HuberRegressor

Xtrain = [train.drop(columns=['DATE','permno','RET']).values,train_bottom.drop(columns=['DATE','permno','RET']).values,train_top.drop(columns=['DATE','permno','RET']).values]
Ytrain = [train['RET'].values,train_bottom['RET'].values,train_top['RET'].values]

Xtest = [test.drop(columns=['DATE','permno','RET']).values,test_bottom.drop(columns=['DATE','permno','RET']).values,test_top.drop(columns=['DATE','permno','RET']).values]
Ytrue = [test['RET'].values,test_bottom['RET'].values,test_top['RET'].values]
i = 0
while i<3:
    model = linear_model.LinearRegression()
    model.fit(X=Xtrain[i],y=Ytrain[i])
    pred = model.predict(X=Xtrain[i])
    epsilon=np.max((np.percentile(np.abs(Ytrain[i]-pred),99.9),1))
    model1 = linear_model.HuberRegressor(epsilon=epsilon,max_iter=1000)
    model1.fit(X=Xtrain[i],y=Ytrain[i])
    pred1 = model1.predict(X=Xtest[i])
    R2 = r2_score(y_true=Ytrue[i],y_pred=pred1)
    R2_OOS = R_oss(true=Ytrue[i],pred=pred1)
    if i == 0:
        print(f'In sample R2 value for complete dataset is: {R2}\n')
        print(f'Out of sample R2 value for complete dataset is: {R2_OOS}\n')
    if i == 1:
        print(f'In sample R2 value for bottom performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for bottom performers of dataset is: {R2_OOS}\n')
    if i == 2:
        print(f'In sample R2 value for top performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for top performers of dataset is: {R2_OOS}\n')
        df = pd.DataFrame({'permno':test_top['permno'],'date':test_top['DATE'],'ret':test_top['RET'],'pred':pred1})
        df.to_csv('OLS_Huber.csv')
    i+=1

/Users/patrickcamp/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[-0.088191  0.079484 -0.015975 ...  0.071545 -0.065069 -0.076855]
[0.35781891 0.20020415 0.40335709 ... 0.38817136 0.39252133 0.54733779]
In sample R2 value for complete dataset is: -2.4180042076933956

Out of sample R2 value for complete dataset is: -2.4075246628045055



/Users/patrickcamp/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[-0.185185 -0.0625   -0.21123  ...  0.157965 -0.171607  0.001463]
[0.52568137 0.59502166 0.50835672 ... 0.56160146 0.57079655 0.51154182]
In sample R2 value for bottom performers of dataset is: -2.224957997248551

Out of sample R2 value for bottom performers of dataset is: -2.2155093696513704



/Users/patrickcamp/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[ 0.074229 -0.068037 -0.001236 ...  0.152268  0.10736   0.010708]
[ 0.19066658  0.15447693  0.17042089 ... -0.05007553  0.04742882
  0.09150361]
In sample R2 value for top performers of dataset is: -1.8829795073700102

Out of sample R2 value for top performers of dataset is: -1.8460126898148541



ValueError: array length 71837 does not match index length 47892

Linear model using OLS and Huber as loss function. Limiting Xtrain datasets to size, book-to-market, and momentum features

In [13]:
Xtrain = [train[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,train_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,train_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Ytrain = [train['RET'].values,train_bottom['RET'].values,train_top['RET'].values]

Xtest = [test[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,test_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,test_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Ytrue = [test['RET'].values,test_bottom['RET'].values,test_top['RET'].values]
i = 0
while i<3:
    model = linear_model.LinearRegression()
    model.fit(X=Xtrain[i],y=Ytrain[i])
    pred = model.predict(X=Xtrain[i])
    epsilon=np.max((np.percentile(np.abs(Ytrain[i]-pred),99.9),1))
    print(f'Epsilon: {epsilon}')
    model1 = linear_model.HuberRegressor(epsilon=epsilon,max_iter=1000)
    model1.fit(X=Xtrain[i],y=Ytrain[i])
    pred1 = model1.predict(X=Xtest[i])
    R2 = r2_score(y_true=Ytrue[i],y_pred=pred1)
    R2_OOS = R_oss(true=Ytrue[i],pred=pred1)
    if i == 0:
        print(f'In sample R2 value for complete dataset is: {R2}\n')
        print(f'Out of sample R2 value for complete dataset is: {R2_OOS}\n')
    if i == 1:
        print(f'In sample R2 value for bottom performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for bottom performers of dataset is: {R2_OOS}\n')
    if i == 2:
        print(f'In sample R2 value for top performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for top performers of dataset is: {R2_OOS}\n')
        df = pd.DataFrame({'permno':test_top['permno'],'date':test_top['DATE'],'ret':test_top['RET'],'pred':pred1})
        df.to_csv('OLS_Huber_3.csv')
    i+=1

Epsilon: 1.0576911130990945
[-0.088191  0.079484 -0.015975 ...  0.071545 -0.065069 -0.076855]
[0.01020608 0.01009598 0.01111188 ... 0.01058256 0.01835172 0.05801039]
In sample R2 value for complete dataset is: -0.0024148641934202963

Out of sample R2 value for complete dataset is: 0.0006585233533125123

Epsilon: 1.9093812923739626
[-0.185185 -0.0625   -0.21123  ...  0.157965 -0.171607  0.001463]
[-0.00280596 -0.01519182 -0.01406333 ... -0.00227047 -0.00227047
 -0.00227047]
In sample R2 value for bottom performers of dataset is: -0.006383516118939614

Out of sample R2 value for bottom performers of dataset is: -0.003434968239599323

Epsilon: 1.0
[ 0.074229 -0.068037 -0.001236 ...  0.152268  0.10736   0.010708]
[-0.00306448 -0.00075989 -0.00110221 ...  0.01882641  0.02191942
  0.01764656]
In sample R2 value for top performers of dataset is: -0.0024828921845239105

Out of sample R2 value for top performers of dataset is: 0.010371379614083964



Linear Model with LASSO regularization, using MSE loss function

In [ ]:
Xtrain = [train.drop(columns=['DATE','permno','RET']).values,train_bottom.drop(columns=['DATE','permno','RET']).values,train_top.drop(columns=['DATE','permno','RET']).values]
Ytrain = [train['RET'].values,train_bottom['RET'].values,train_top['RET'].values]

Xtest = [test.drop(columns=['DATE','permno','RET']).values,test_bottom.drop(columns=['DATE','permno','RET']).values,test_top.drop(columns=['DATE','permno','RET']).values]
Ytrue = [test['RET'].values,test_bottom['RET'].values,test_top['RET'].values]

Xval = [val.drop(columns=['DATE','permno','RET']).values,val_bottom.drop(columns=['DATE','permno','RET']).values,val_top.drop(columns=['DATE','permno','RET']).values]
Yval = [val['RET'].values,val_bottom['RET'].values,val_top['RET'].values]

i = 0
alphas = 10**np.linspace(-6,6,10)

while i<3:
    best_mse = np.inf
    best_alpha = 0
    for alpha in alphas:
        model = Lasso(alpha=alpha,max_iter=1000)
        model.fit(X=Xtrain[i],y=Ytrain[i])
        y_predVal = model.predict(X=Xval[i])
        mse = mean_squared_error(y_true=Yval[i],y_pred=y_predVal)
        if mse < best_mse:
            best_mse = mse
            best_alpha = alpha
    model = Lasso(alpha=best_alpha,max_iter=1000)
    print(f'best_alpha: {best_alpha}')
    model.fit(X=Xtrain[i],y=Ytrain[i])
    y_pred = model.predict(X=Xtest[i])
    R2_OOS = R_oss(true=Ytrue[i],pred=y_pred)
    if i == 0:
        print(f'In sample R2 value for complete dataset is: {R2}\n')
        print(f'Out of sample R2 value for complete dataset is: {R2_OOS}\n')
    if i == 1:
        print(f'In sample R2 value for bottom performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for bottom performers of dataset is: {R2_OOS}\n')
    if i == 2:
        print(f'In sample R2 value for top performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for top performers of dataset is: {R2_OOS}\n')
        df = pd.DataFrame({'permno':test_top['permno'],'date':test_top['DATE'],'ret':test_top['RET'],'pred':y_pred})
        df.to_csv('OLS_LASSO.csv')
    i+=1

Linear Model with LASSO regularization, using MSE as loss function. Limiting Xtrain to size, book-to-market, and momentum

In [14]:
Xtrain = [train[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,train_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,train_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Ytrain = [train['RET'].values,train_bottom['RET'].values,train_top['RET'].values]

Xtest = [test[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,test_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,test_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Ytrue = [test['RET'].values,test_bottom['RET'].values,test_top['RET'].values]

Xval = [val[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,val_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,val_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Yval = [val['RET'].values,val_bottom['RET'].values,val_top['RET'].values]

i = 0
alphas = 10**np.linspace(-6,6,100)

while i<3:
    best_mse = np.inf
    best_alpha = 0
    for alpha in alphas:
        model = Lasso(alpha=alpha,max_iter=1000)
        model.fit(X=Xtrain[i],y=Ytrain[i])
        y_predVal = model.predict(X=Xval[i])
        mse = mean_squared_error(y_true=Yval[i],y_pred=y_predVal)
        if mse < best_mse:
            best_mse = mse
            best_alpha = alpha
    print(f'best_alpha: {best_alpha}')
    model = Lasso(alpha=best_alpha,max_iter=1000)
    model.fit(X=Xtrain[i],y=Ytrain[i])
    y_pred = model.predict(X=Xtest[i])
    R2_OOS = R_oss(true=Ytrue[i],pred=y_pred)
    if i == 0:
        print(f'In sample R2 value for complete dataset is: {R2}\n')
        print(f'Out of sample R2 value for complete dataset is: {R2_OOS}\n')
    if i == 1:
        print(f'In sample R2 value for bottom performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for bottom performers of dataset is: {R2_OOS}\n')
    if i == 2:
        print(f'In sample R2 value for top performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for top performers of dataset is: {R2_OOS}\n')
        df = pd.DataFrame({'permno':test_top['permno'],'date':test_top['DATE'],'ret':test_top['RET'],'pred':y_pred})
        df.to_csv('OLS_LASSO_3.csv')
    i+=1

best_alpha: 0.0003511191734215131
[-0.088191  0.079484 -0.015975 ...  0.071545 -0.065069 -0.076855]
[0.0085248  0.0089159  0.00877782 ... 0.00788056 0.01122907 0.00932202]
In sample R2 value for complete dataset is: -0.0024828921845239105

Out of sample R2 value for complete dataset is: 0.0028276894578835865

best_alpha: 0.0004641588833612782
[-0.185185 -0.0625   -0.21123  ...  0.157965 -0.171607  0.001463]
[-0.0120389  -0.01024036 -0.00982016 ...  0.00978743  0.00978743
  0.00978743]
In sample R2 value for bottom performers of dataset is: -0.0024828921845239105

Out of sample R2 value for bottom performers of dataset is: 0.0033709203583318637

best_alpha: 0.0004641588833612782
[ 0.074229 -0.068037 -0.001236 ...  0.152268  0.10736   0.010708]
[0.01032872 0.01032872 0.01032872 ... 0.01032872 0.01032872 0.01032872]
In sample R2 value for top performers of dataset is: -0.0024828921845239105

Out of sample R2 value for top performers of dataset is: 0.01261617282813876



Linear Model with LASSO regularization, using Huber loss function

In [ ]:
Xtrain = [train.drop(columns=['DATE','permno','RET']).values,train_bottom.drop(columns=['DATE','permno','RET']).values,train_top.drop(columns=['DATE','permno','RET']).values]
Ytrain = [train['RET'].values,train_bottom['RET'].values,train_top['RET'].values]

Xtest = [test.drop(columns=['DATE','permno','RET']).values,test_bottom.drop(columns=['DATE','permno','RET']).values,test_top.drop(columns=['DATE','permno','RET']).values]
Ytrue = [test['RET'].values,test_bottom['RET'].values,test_top['RET'].values]

Xval = [val.drop(columns=['DATE','permno','RET']).values,val_bottom.drop(columns=['DATE','permno','RET']).values,val_top.drop(columns=['DATE','permno','RET']).values]
Yval = [val['RET'].values,val_bottom['RET'].values,val_top['RET'].values]

i = 0
gammas = 10**np.linspace(-6,6,10)
lamdas = 10**np.linspace(-6,6,10)

while i<3:
    best_mse = np.inf
    best_gamma=0
    best_lamda=0
    model = linear_model.LinearRegression()
    model.fit(X=Xtrain[i],y=Ytrain[i])
    pred = model.predict(X=Xtrain[i])
    epsilon=np.max((np.percentile(np.abs(Ytrain[i]-pred),99.9),1))
    print(f'epsilon: {epsilon}')
    for gamma in gammas:
        for lamda in lamdas:
            theta = prox_grad(gamma=gamma,lamda=lamda,X=Xtrain[i],y=Ytrain[i],model='lasso',loss_type='huber',epsilon=epsilon)
            pred = fit_APG(Xtest=Xval[i],theta=theta)
            loss = HuberLoss(true=Yval[i],pred=pred,epsilon=epsilon)
            if loss < best_mse:
                best_mse = loss
                best_gamma = gamma
                best_lamda = lamda
    print(f'best gamma: {best_gamma}\nbest lamda: {best_lamda}')
    theta = prox_grad(gamma=best_gamma,lamda=best_lamda,X=Xtrain[i],y=Ytrain[i],model='lasso',loss_type='huber',epsilon=epsilon)
    pred = fit_APG(Xtest=Xtest[i],theta=theta)
    R2_OOS = R_oss(true=Ytrue[i],pred=pred)
    if i == 0:
        print(f'In sample R2 value for complete dataset is: {R2}\n')
        print(f'Out of sample R2 value for complete dataset is: {R2_OOS}\n')
    if i == 1:
        print(f'In sample R2 value for bottom performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for bottom performers of dataset is: {R2_OOS}\n')
    if i == 2:
        print(f'In sample R2 value for top performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for top performers of dataset is: {R2_OOS}\n')
    i+=1

Linear Model with LASSO regularization, using Huber loss function. Limitng X dataset to size, book-to-market, and momentum.

In [8]:
Xtrain = [train[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,train_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,train_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Ytrain = [train['RET'].values,train_bottom['RET'].values,train_top['RET'].values]

Xtest = [test[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,test_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,test_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Ytrue = [test['RET'].values,test_bottom['RET'].values,test_top['RET'].values]

Xval = [val[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,val_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,val_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Yval = [val['RET'].values,val_bottom['RET'].values,val_top['RET'].values]

i = 0
gammas = 10**np.linspace(-6,6,10)
lamdas = 10**np.linspace(-6,6,10)

while i<3:
    best_mse = np.inf
    best_gamma=0
    best_lamda=0
    model = linear_model.LinearRegression()
    model.fit(X=Xtrain[i],y=Ytrain[i])
    pred = model.predict(X=Xtrain[i])
    epsilon=np.max((np.percentile(np.abs(Ytrain[i]-pred),99.9),1))
    print(f'epsilon: {epsilon}')
    for gamma in gammas:
        for lamda in lamdas:
            theta = prox_grad(gamma=gamma,lamda=lamda,X=Xtrain[i],y=Ytrain[i],model='lasso',loss_type='huber',epsilon=epsilon)
            pred = fit_APG(Xtest=Xval[i],theta=theta)
            loss = HuberLoss(true=Yval[i],pred=pred,epsilon=epsilon)
            if loss < best_mse:
                best_mse = loss
                best_gamma = gamma
                best_lamda = lamda
    print(f'best gamma: {best_gamma}\nbest lamda: {best_lamda}')
    print(loss)
    theta = prox_grad(gamma=best_gamma,lamda=best_lamda,X=Xtrain[i],y=Ytrain[i],model='lasso',loss_type='huber',epsilon=epsilon)
    pred = fit_APG(Xtest=Xtest[i],theta=theta)
    R2_OOS = R_oss(true=Ytrue[i],pred=pred)
    if i == 0:
        print(f'Out of sample R2 value for complete dataset is: {R2_OOS}\n')
    if i == 1:
        print(f'Out of sample R2 value for bottom performers of dataset is: {R2_OOS}\n')
    if i == 2:
        print(f'Out of sample R2 value for top performers of dataset is: {R2_OOS}\n')
    i+=1

epsilon: 1.0576911130990945
best gamma: 100.0
best lamda: 1e-06
0.020156711366922694
[-0.088191  0.079484 -0.015975 ...  0.071545 -0.065069 -0.076855]
[1.86568174e-05 1.99576846e-05 1.85168470e-05 ... 1.82145396e-05
 1.91744894e-05 4.71208837e-06]
Out of sample R2 value for complete dataset is: 1.0403974521122628e-05

epsilon: 1.9093812923739626
best gamma: 1e-06
best lamda: 0.01
0.05137084521349965
[-0.185185 -0.0625   -0.21123  ...  0.157965 -0.171607  0.001463]
[0. 0. 0. ... 0. 0. 0.]
Out of sample R2 value for bottom performers of dataset is: 0.0

epsilon: 1.0
best gamma: 1e-06
best lamda: 1e-06
0.005790145392873892
[ 0.074229 -0.068037 -0.001236 ...  0.152268  0.10736   0.010708]
[0. 0. 0. ... 0. 0. 0.]
Out of sample R2 value for top performers of dataset is: 0.0



Linear Model with Ridge regularization, using MSE loss function

In [ ]:
Xtrain = [train.drop(columns=['DATE','permno','RET']).values,train_bottom.drop(columns=['DATE','permno','RET']).values,train_top.drop(columns=['DATE','permno','RET']).values]
Ytrain = [train['RET'].values,train_bottom['RET'].values,train_top['RET'].values]

Xtest = [test.drop(columns=['DATE','permno','RET']).values,test_bottom.drop(columns=['DATE','permno','RET']).values,test_top.drop(columns=['DATE','permno','RET']).values]
Ytrue = [test['RET'].values,test_bottom['RET'].values,test_top['RET'].values]

Xval = [val.drop(columns=['DATE','permno','RET']).values,val_bottom.drop(columns=['DATE','permno','RET']).values,val_top.drop(columns=['DATE','permno','RET']).values]
Yval = [val['RET'].values,val_bottom['RET'].values,val_top['RET'].values]

i = 0
alphas = 10**np.linspace(-6,6,10)

while i<3:
    best_mse = np.inf
    best_alpha = 0
    for alpha in alphas:
        model = Ridge(alpha=alpha,max_iter=1000)
        model.fit(X=Xtrain[i],y=Ytrain[i])
        y_predVal = model.predict(X=Xval[i])
        mse = mean_squared_error(y_true=Yval[i],y_pred=y_predVal)
        if mse < best_mse:
            best_mse = mse
            best_alpha = alpha
    print(f'best_alpha: {best_alpha}')
    model = Ridge(alpha=best_alpha,max_iter=1000)
    model.fit(X=Xtrain[i],y=Ytrain[i])
    y_pred = model.predict(X=Xtest[i])
    R2_OOS = R_oss(true=Ytrue[i],pred=y_pred)
    if i == 0:
        print(f'In sample R2 value for complete dataset is: {R2}\n')
        print(f'Out of sample R2 value for complete dataset is: {R2_OOS}\n')
    if i == 1:
        print(f'In sample R2 value for bottom performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for bottom performers of dataset is: {R2_OOS}\n')
    if i == 2:
        print(f'In sample R2 value for top performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for top performers of dataset is: {R2_OOS}\n')
    i+=1

Linear Model with Ridge regularization, using MSE as loss function. Limiting Xtrain to size, mook-to-market, and momentum.

In [15]:
Xtrain = [train[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,train_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,train_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Ytrain = [train['RET'].values,train_bottom['RET'].values,train_top['RET'].values]

Xtest = [test[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,test_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,test_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Ytrue = [test['RET'].values,test_bottom['RET'].values,test_top['RET'].values]

Xval = [val[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,val_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,val_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Yval = [val['RET'].values,val_bottom['RET'].values,val_top['RET'].values]

i = 0
alphas = 10**np.linspace(-6,6,100)

while i<3:
    best_mse = np.inf
    best_alpha = 0
    for alpha in alphas:
        model = Ridge(alpha=alpha,max_iter=1000)
        model.fit(X=Xtrain[i],y=Ytrain[i])
        y_predVal = model.predict(X=Xval[i])
        mse = mean_squared_error(y_true=Yval[i],y_pred=y_predVal)
        if mse < best_mse:
            best_mse = mse
            best_alpha = alpha
    print(f'best_alpha: {best_alpha}')
    model = Ridge(alpha=best_alpha,max_iter=1000)
    model.fit(X=Xtrain[i],y=Ytrain[i])
    y_pred = model.predict(X=Xtest[i])
    R2_OOS = R_oss(true=Ytrue[i],pred=y_pred)
    if i == 0:
        print(f'In sample R2 value for complete dataset is: {R2}\n')
        print(f'Out of sample R2 value for complete dataset is: {R2_OOS}\n')
    if i == 1:
        print(f'In sample R2 value for bottom performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for bottom performers of dataset is: {R2_OOS}\n')
    if i == 2:
        print(f'In sample R2 value for top performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for top performers of dataset is: {R2_OOS}\n')
        df = pd.DataFrame({'permno':test_top['permno'],'date':test_top['DATE'],'ret':test_top['RET'],'pred':y_pred})
        df.to_csv('OLS_Ridge_3.csv')
    i+=1

best_alpha: 46415.888336127915
[-0.088191  0.079484 -0.015975 ...  0.071545 -0.065069 -0.076855]
[0.00857403 0.0090085  0.0087027  ... 0.00816591 0.0102446  0.00787592]
In sample R2 value for complete dataset is: -0.0024828921845239105

Out of sample R2 value for complete dataset is: 0.0028872231080153687

best_alpha: 2848.035868435805
[-0.185185 -0.0625   -0.21123  ...  0.157965 -0.171607  0.001463]
[-0.01130761 -0.00815716 -0.00824147 ...  0.01103922  0.01103922
  0.01103922]
In sample R2 value for bottom performers of dataset is: -0.0024828921845239105

Out of sample R2 value for bottom performers of dataset is: 0.0034148308470604016

best_alpha: 1000000.0
[ 0.074229 -0.068037 -0.001236 ...  0.152268  0.10736   0.010708]
[0.01031627 0.01032001 0.01031745 ... 0.01033639 0.01034118 0.01033548]
In sample R2 value for top performers of dataset is: -0.0024828921845239105

Out of sample R2 value for top performers of dataset is: 0.012619944371153324



Linear Model with Ridge regularization, using Huber loss function

In [ ]:
Xtrain = [train.drop(columns=['DATE','permno','RET']).values,train_bottom.drop(columns=['DATE','permno','RET']).values,train_top.drop(columns=['DATE','permno','RET']).values]
Ytrain = [train['RET'].values,train_bottom['RET'].values,train_top['RET'].values]

Xtest = [test.drop(columns=['DATE','permno','RET']).values,test_bottom.drop(columns=['DATE','permno','RET']).values,test_top.drop(columns=['DATE','permno','RET']).values]
Ytrue = [test['RET'].values,test_bottom['RET'].values,test_top['RET'].values]

Xval = [val.drop(columns=['DATE','permno','RET']).values,val_bottom.drop(columns=['DATE','permno','RET']).values,val_top.drop(columns=['DATE','permno','RET']).values]
Yval = [val['RET'].values,val_bottom['RET'].values,val_top['RET'].values]

i = 0
gammas = 10**np.linspace(-6,6,10)
lamdas = 10**np.linspace(-6,6,10)

while i<3:
    best_mse = np.inf
    best_gamma=0
    best_lamda=0
    model = linear_model.LinearRegression()
    model.fit(X=Xtrain[i],y=Ytrain[i])
    pred = model.predict(X=Xtrain[i])
    epsilon=np.max((np.percentile(np.abs(Ytrain[i]-pred),99.9),1))
    print(f'epsilon: {epsilon}')
    for gamma in gammas:
        for lamda in lamdas:
            theta = prox_grad(gamma=gamma,lamda=lamda,X=Xtrain[i],y=Ytrain[i],model='ridge',loss_type='huber',epsilon=epsilon)
            pred = fit_APG(Xtest=Xval[i],theta=theta)
            loss = HuberLoss(true=Yval[i],pred=pred,epsilon=epsilon)
            if loss < best_mse:
                best_mse = loss
                best_gamma = gamma
                best_lamda = lamda
    print(f'best gamma: {best_gamma}\nbest lamda: {best_lamda}')
    theta = prox_grad(gamma=best_gamma,lamda=best_lamda,X=Xtrain[i],y=Ytrain[i],model='ridge',loss_type='huber',epsilon=epsilon)
    pred = fit_APG(Xtest=Xtest[i],theta=theta)
    R2_OOS = R_oss(true=Ytrue[i],pred=pred)
    if i == 0:
        print(f'In sample R2 value for complete dataset is: {R2}\n')
        print(f'Out of sample R2 value for complete dataset is: {R2_OOS}\n')
    if i == 1:
        print(f'In sample R2 value for bottom performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for bottom performers of dataset is: {R2_OOS}\n')
    if i == 2:
        print(f'In sample R2 value for top performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for top performers of dataset is: {R2_OOS}\n')
    i+=1

Linear Model with Ridge regularization, using Huber loss function. Limiting X dataset to size, book-to-market, and momentum

In [13]:
Xtrain = [train[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,train_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,train_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Ytrain = [train['RET'].values,train_bottom['RET'].values,train_top['RET'].values]

Xtest = [test[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,test_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,test_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Ytrue = [test['RET'].values,test_bottom['RET'].values,test_top['RET'].values]

Xval = [val[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,val_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,val_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Yval = [val['RET'].values,val_bottom['RET'].values,val_top['RET'].values]

i = 0
gammas = 10**np.linspace(-6,6,10)
lamdas = 10**np.linspace(-6,6,10)

while i<3:
    best_mse = np.inf
    best_gamma=0
    best_lamda=0
    model = linear_model.LinearRegression()
    model.fit(X=Xtrain[i],y=Ytrain[i])
    pred = model.predict(X=Xtrain[i])
    epsilon=np.max((np.percentile(np.abs(Ytrain[i]-pred),99.9),1))
    print(f'epsilon: {epsilon}')
    for gamma in gammas:
        for lamda in lamdas:
            theta = prox_grad(gamma=gamma,lamda=lamda,X=Xtrain[i],y=Ytrain[i],model='ridge',loss_type='huber',epsilon=epsilon)
            pred = fit_APG(Xtest=Xval[i],theta=theta)
            loss = HuberLoss(true=Yval[i],pred=pred,epsilon=epsilon)
            if loss < best_mse:
                best_mse = loss
                best_gamma = gamma
                best_lamda = lamda
    print(f'best gamma: {best_gamma}\nbest lamda: {best_lamda}')
    theta = prox_grad(gamma=best_gamma,lamda=best_lamda,X=Xtrain[i],y=Ytrain[i],model='ridge',loss_type='huber',epsilon=epsilon)
    pred = fit_APG(Xtest=Xtest[i],theta=theta)
    R2_OOS = R_oss(true=Ytrue[i],pred=pred)
    if i == 0:
        print(f'In sample R2 value for complete dataset is: {R2}\n')
        print(f'Out of sample R2 value for complete dataset is: {R2_OOS}\n')
    if i == 1:
        print(f'In sample R2 value for bottom performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for bottom performers of dataset is: {R2_OOS}\n')
    if i == 2:
        print(f'In sample R2 value for top performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for top performers of dataset is: {R2_OOS}\n')
    i+=1

epsilon: 1.0576911130990945
best gamma: 0.00046415888336127773
best lamda: 1000000.0
[-0.088191  0.079484 -0.015975 ...  0.071545 -0.065069 -0.076855]
[-8.87700998e-09 -9.86978773e-09 -8.85303344e-09 ... -8.48446420e-09
 -9.55011094e-09 -2.78550912e-09]
In sample R2 value for complete dataset is: -0.0024828921845239105

Out of sample R2 value for complete dataset is: -4.982344092852031e-09

epsilon: 1.9093812923739626
best gamma: 0.00046415888336127773
best lamda: 1000000.0
[-0.185185 -0.0625   -0.21123  ...  0.157965 -0.171607  0.001463]
[-5.38309243e-09 -7.50872042e-09 -7.06923518e-09 ... -1.12671837e-08
 -1.12671837e-08 -1.12671837e-08]
In sample R2 value for bottom performers of dataset is: -0.0024828921845239105

Out of sample R2 value for bottom performers of dataset is: -3.9465171131070065e-09

epsilon: 1.0
best gamma: 1e-06
best lamda: 1e-06
[ 0.074229 -0.068037 -0.001236 ...  0.152268  0.10736   0.010708]
[0. 0. 0. ... 0. 0. 0.]
In sample R2 value for top performers of dataset

Linear Model with Elastic Net regularization, using MSE loss function

In [ ]:
Xtrain = [train.drop(columns=['DATE','permno','RET']).values,train_bottom.drop(columns=['DATE','permno','RET']).values,train_top.drop(columns=['DATE','permno','RET']).values]
Ytrain = [train['RET'].values,train_bottom['RET'].values,train_top['RET'].values]

Xtest = [test.drop(columns=['DATE','permno','RET']).values,test_bottom.drop(columns=['DATE','permno','RET']).values,test_top.drop(columns=['DATE','permno','RET']).values]
Ytrue = [test['RET'].values,test_bottom['RET'].values,test_top['RET'].values]

Xval = [val.drop(columns=['DATE','permno','RET']).values,val_bottom.drop(columns=['DATE','permno','RET']).values,val_top.drop(columns=['DATE','permno','RET']).values]
Yval = [val['RET'].values,val_bottom['RET'].values,val_top['RET'].values]

alphas = 10**np.linspace(-6,6,10)
l1_ratios = 10**np.linspace(-6,0,10)

i=0
while i < 3:
    best_mse = 0
    best_alpha = 0
    best_l1_ratio = 0
    for alpha in alphas:
        for l1_ratio in l1_ratios:
            model = ElasticNet(alpha=alpha,l1_ratio=l1_ratio,max_iter=1000)
            model.fit(X=Xtrain[i],y=Ytrain[i])
            y_predVal = model.predict(Xval[i])
            mse = mean_squared_error(y_true=Yval[i],y_pred=y_predVal)
            if mse < best_mse:
                best_mse = mse
                best_alpha = alpha
                best_l1_ratio = l1_ratio
    print(f'best alpha: {best_alpha}\nbest l1_ratio: {best_l1_ratio}')
    model = ElasticNet(alpha=best_alpha,l1_ratio=best_l1_ratio,max_iter=1000)
    model.fit(X=Xtrain[i],y=Ytrain[i])
    y_pred = model.predict(X=Xtest[i])
    R2_OOS = R_oss(true=Ytrue[i],pred=y_pred)
    if i == 0:
        print(f'In sample R2 value for complete dataset is: {R2}\n')
        print(f'Out of sample R2 value for complete dataset is: {R2_OOS}\n')
    if i == 1:
        print(f'In sample R2 value for bottom performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for bottom performers of dataset is: {R2_OOS}\n')
    if i == 2:
        print(f'In sample R2 value for top performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for top performers of dataset is: {R2_OOS}\n')
    i+=1

Linear Model with Elastic Net regularization, using MSE as loss function. Limiting X datasets to size, book-to-market, and momentum.

In [16]:
Xtrain = [train[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,train_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,train_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Ytrain = [train['RET'].values,train_bottom['RET'].values,train_top['RET'].values]

Xtest = [test[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,test_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,test_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Ytrue = [test['RET'].values,test_bottom['RET'].values,test_top['RET'].values]

Xval = [val[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,val_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,val_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Yval = [val['RET'].values,val_bottom['RET'].values,val_top['RET'].values]

alphas = 10**np.linspace(-6,6,100)
l1_ratios = 10**np.linspace(-6,0,100)
i=0
while i < 3:
    best_mse = np.inf
    best_alpha = 0
    best_l1_ratio = 0
    for alpha in alphas:
        for l1_ratio in l1_ratios:
            model = ElasticNet(alpha=alpha,l1_ratio=l1_ratio,max_iter=1000)
            model.fit(X=Xtrain[i],y=Ytrain[i])
            y_predVal = model.predict(Xval[i])
            mse = mean_squared_error(y_true=Yval[i],y_pred=y_predVal)
            if mse < best_mse:
                best_mse = mse
                best_alpha = alpha
                best_l1_ratio = l1_ratio
    print(f'best alpha: {best_alpha}\nbest l1_ratio: {best_l1_ratio}')
    model = ElasticNet(alpha=best_alpha,l1_ratio=best_l1_ratio,max_iter=1000)
    model.fit(X=Xtrain[i],y=Ytrain[i])
    y_pred = model.predict(X=Xtest[i])
    R2_OOS = R_oss(true=Ytrue[i],pred=y_pred)
    if i == 0:
        print(f'In sample R2 value for complete dataset is: {R2}\n')
        print(f'Out of sample R2 value for complete dataset is: {R2_OOS}\n')
    if i == 1:
        print(f'In sample R2 value for bottom performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for bottom performers of dataset is: {R2_OOS}\n')
    if i == 2:
        print(f'In sample R2 value for top performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for top performers of dataset is: {R2_OOS}\n')
        df = pd.DataFrame({'permno':test_top['permno'],'date':test_top['DATE'],'ret':test_top['RET'],'pred':y_pred})
        df.to_csv('OLS_ENet_3.csv')
    i+=1

best alpha: 0.07054802310718646
best l1_ratio: 0.0014174741629268048
[-0.088191  0.079484 -0.015975 ...  0.071545 -0.065069 -0.076855]
[0.00860455 0.00891386 0.00878506 ... 0.00811736 0.01050056 0.00910094]
In sample R2 value for complete dataset is: -0.0024828921845239105

Out of sample R2 value for complete dataset is: 0.0028713563564116695

best alpha: 0.030538555088334186
best l1_ratio: 0.002848035868435802
[-0.185185 -0.0625   -0.21123  ...  0.157965 -0.171607  0.001463]
[-0.01141986 -0.00868664 -0.00882123 ...  0.01110221  0.01110221
  0.01110221]
In sample R2 value for bottom performers of dataset is: -0.0024828921845239105

Out of sample R2 value for bottom performers of dataset is: 0.0034253929172095576

best alpha: 0.0004641588833612782
best l1_ratio: 1.0
[ 0.074229 -0.068037 -0.001236 ...  0.152268  0.10736   0.010708]
[0.01032872 0.01032872 0.01032872 ... 0.01032872 0.01032872 0.01032872]
In sample R2 value for top performers of dataset is: -0.0024828921845239105

Out of sa

Linear Model with Elastic Net reuglarization, using Huber loss function

In [ ]:
Xtrain = [train.drop(columns=['DATE','permno','RET']).values,train_bottom.drop(columns=['DATE','permno','RET']).values,train_top.drop(columns=['DATE','permno','RET']).values]
Ytrain = [train['RET'].values,train_bottom['RET'].values,train_top['RET'].values]

Xtest = [test.drop(columns=['DATE','permno','RET']).values,test_bottom.drop(columns=['DATE','permno','RET']).values,test_top.drop(columns=['DATE','permno','RET']).values]
Ytrue = [test['RET'].values,test_bottom['RET'].values,test_top['RET'].values]

Xval = [val.drop(columns=['DATE','permno','RET']).values,val_bottom.drop(columns=['DATE','permno','RET']).values,val_top.drop(columns=['DATE','permno','RET']).values]
Yval = [val['RET'].values,val_bottom['RET'].values,val_top['RET'].values]

i = 0
gammas = 10**np.linspace(-6,6,10)
lamdas = 10**np.linspace(-6,6,10)
l1_ratios = 10**np.linspace(-6,0,10)

while i<3:
    best_mse = np.inf
    best_gamma=0
    best_lamda=0
    best_l1_ratio = 0
    model = linear_model.LinearRegression()
    model.fit(X=Xtrain[i],y=Ytrain[i])
    pred = model.predict(X=Xtrain[i])
    epsilon=np.max((np.percentile(np.abs(Ytrain[i]-pred),99.9),1))
    for gamma in gammas:
        for lamda in lamdas:
            for rho in l1_ratios:
                theta = prox_grad(gamma=gamma,lamda=lamda,X=Xtrain[i],y=Ytrain[i],model='elastic',loss_type='huber',epsilon=epsilon,rho=rho)
                pred = fit_APG(Xtest=Xval[i],theta=theta)
                loss = HuberLoss(true=Yval[i],pred=pred,epsilon=epsilon)
                if loss < best_mse:
                    best_mse = loss
                    best_gamma = gamma
                    best_lamda = lamda
                    best_l1_ratio = rho
    theta = prox_grad(gamma=best_gamma,lamda=best_lamda,X=Xtrain[i],y=Ytrain[i],model='elastic',loss_type='huber',epsilon=epsilon,rho=best_l1_ratio)
    pred = fit_APG(Xtest=Xtest[i],theta=theta)
    R2_OOS = R_oss(true=Ytrue[i],pred=pred)
    if i == 0:
        print(f'In sample R2 value for complete dataset is: {R2}\n')
        print(f'Out of sample R2 value for complete dataset is: {R2_OOS}\n')
    if i == 1:
        print(f'In sample R2 value for bottom performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for bottom performers of dataset is: {R2_OOS}\n')
    if i == 2:
        print(f'In sample R2 value for top performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for top performers of dataset is: {R2_OOS}\n')
    i+=1

Linear Model with Elastic Net regularization, using Huber loss function. Limiting X dataset to size, book-to-market, momentum. 

In [15]:
Xtrain = [train[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,train_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,train_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Ytrain = [train['RET'].values,train_bottom['RET'].values,train_top['RET'].values]

Xtest = [test[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,test_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,test_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Ytrue = [test['RET'].values,test_bottom['RET'].values,test_top['RET'].values]

Xval = [val[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,val_bottom[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values,val_top[['mvel1','bm','mom1m','mom6m','mom12m','mom36m']].values]
Yval = [val['RET'].values,val_bottom['RET'].values,val_top['RET'].values]

i = 0
gammas = 10**np.linspace(-6,6,10)
lamdas = 10**np.linspace(-6,6,10)
l1_ratios = 10**np.linspace(-6,0,10)

while i<3:
    best_mse = np.inf
    best_gamma=0
    best_lamda=0
    best_l1_ratio = 0
    model = linear_model.LinearRegression()
    model.fit(X=Xtrain[i],y=Ytrain[i])
    pred = model.predict(X=Xtrain[i])
    epsilon=np.max((np.percentile(np.abs(Ytrain[i]-pred),99.9),1))
    for gamma in gammas:
        for lamda in lamdas:
            for rho in l1_ratios:
                theta = prox_grad(gamma=gamma,lamda=lamda,X=Xtrain[i],y=Ytrain[i],model='elastic',loss_type='huber',epsilon=epsilon,rho=rho)
                pred = fit_APG(Xtest=Xval[i],theta=theta)
                loss = HuberLoss(true=Yval[i],pred=pred,epsilon=epsilon)
                if loss < best_mse:
                    best_mse = loss
                    best_gamma = gamma
                    best_lamda = lamda
                    best_l1_ratio = rho
    theta = prox_grad(gamma=best_gamma,lamda=best_lamda,X=Xtrain[i],y=Ytrain[i],model='elastic',loss_type='huber',epsilon=epsilon,rho=best_l1_ratio)
    pred = fit_APG(Xtest=Xtest[i],theta=theta)
    R2_OOS = R_oss(true=Ytrue[i],pred=pred)
    if i == 0:
        print(f'In sample R2 value for complete dataset is: {R2}\n')
        print(f'Out of sample R2 value for complete dataset is: {R2_OOS}\n')
    if i == 1:
        print(f'In sample R2 value for bottom performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for bottom performers of dataset is: {R2_OOS}\n')
    if i == 2:
        print(f'In sample R2 value for top performers of dataset is: {R2}\n')
        print(f'Out of sample R2 value for top performers of dataset is: {R2_OOS}\n')
    i+=1

[-0.088191  0.079484 -0.015975 ...  0.071545 -0.065069 -0.076855]
[0. 0. 0. ... 0. 0. 0.]
In sample R2 value for complete dataset is: -0.0024828921845239105

Out of sample R2 value for complete dataset is: 0.0

[-0.185185 -0.0625   -0.21123  ...  0.157965 -0.171607  0.001463]
[0. 0. 0. ... 0. 0. 0.]
In sample R2 value for bottom performers of dataset is: -0.0024828921845239105

Out of sample R2 value for bottom performers of dataset is: 0.0

[ 0.074229 -0.068037 -0.001236 ...  0.152268  0.10736   0.010708]
[0. 0. 0. ... 0. 0. 0.]
In sample R2 value for top performers of dataset is: -0.0024828921845239105

Out of sample R2 value for top performers of dataset is: 0.0

